<a href="https://colab.research.google.com/github/skondaparthi/FinSent-Agent/blob/main/FinSent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FinSent Agent - Financial Sentiment Analysis

This notebook analyzes financial news sentiment using FinBERT and creates embeddings for similarity search.

## Setup Instructions

### For Google Colab:
Run the installation cells below to install required packages.

### For Local Environment:
1. Create a virtual environment: `python -m venv finsentenv`
2. Activate it: `source finsentenv/bin/activate` (Mac/Linux) or `finsentenv\Scripts\activate` (Windows)
3. Install dependencies: `pip install -r requirements.txt`
4. Then you can skip the installation cells and run the analysis directly.

In [ ]:
# For Colab: Install all dependencies from requirements.txt
# For local: Skip this cell if you've already installed via pip install -r requirements.txt
!pip install -r requirements.txt

In [2]:
from google.colab import userdata

PAT = userdata.get('githubPAT')

In [ ]:
!git clone https://{PAT}@github.com/skondaparthi/FinSent-Agent.git

%cd FinSent-Agent

from ingest.news_api import find_news
import models.finbert_model as finbert_model

username = userdata.get("gitUsername")
email = userdata.get("gitEmail")

%shell git config --global user.name "{username}"
%shell git config --global user.email "{email}"


In [7]:
from importlib import reload

# Import the module first
import ingest.news_api

# Now you can reload it
reload(ingest.news_api)

from ingest.news_api import find_news

In [1]:
!git status

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   FinSent.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.gitignore
	models/faiss_model.py

no changes added to commit (use "git add" and/or "git commit -a")


In [63]:
!git add .

In [64]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   ingest/__pycache__/news_api.cpython-311.pyc
	modified:   ingest/news_api.py
	modified:   models/__pycache__/finbert_model.cpython-311.pyc



In [69]:
!git commit -m "added embeddings to df"

[main 1d20c3f] added embeddings to df
 3 files changed, 1 insertion(+), 1 deletion(-)


In [ ]:
!git push https://{PAT}@github.com/skondaparthi/FinSent-Agent.git main

In [6]:
from ingest.news_api import find_news
import models.finbert_model as finbert_model

news_df = find_news()

display(news_df.head())

ModuleNotFoundError: No module named 'transformers'

In [9]:
news_df = finbert_model.score_df(news_df)
news_df.head()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


,source,title,description,publishedAt,sent_scores,category
0,The Verge,Zuckerberg’s ‘personal superintelligence’ plan...,It has been another busy week. GPT-5 appears t...,2025-08-02T00:11:36Z,"{'negative': 0.061627395, 'neutral': 0.0202904...",positive
1,Gizmodo.com,Wall Street’s AI Bubble Is Worse Than the 1999...,A chief economist at investment giant Apollo s...,2025-07-17T13:00:10Z,"{'negative': 0.025278883, 'neutral': 0.7703285...",neutral
2,Gizmodo.com,Truth Social Needs Its Own Grok. Trump Has the...,A MAGA-bot could be on the way.,2025-07-17T13:15:25Z,"{'negative': 0.06825258, 'neutral': 0.04631966...",positive
3,Gizmodo.com,"As Tesla’s Sales Dwindle, a Critical Revenue S...",Tesla needs all the help it can get.,2025-07-23T13:10:27Z,"{'negative': 0.23368655, 'neutral': 0.01441254...",positive
4,Gizmodo.com,Studies Show AI Models Love to Share With One ...,Sharing is caring.,2025-07-30T18:10:01Z,"{'negative': 0.08102807, 'neutral': 0.02253392...",positive


In [1]:
news_df["embeds"] = finbert_model.embed((news_df["title"] + " " +
                                        news_df["description"]).tolist()).tolist()
news_df.head()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


,source,title,description,publishedAt,sent_scores,category,embeds
0,The Verge,Zuckerberg’s ‘personal superintelligence’ plan...,It has been another busy week. GPT-5 appears t...,2025-08-02T00:11:36Z,"{'negative': 0.061627395, 'neutral': 0.0202904...",positive,"[-0.0854862853884697, -0.029945576563477516, 0..."
1,Gizmodo.com,Wall Street’s AI Bubble Is Worse Than the 1999...,A chief economist at investment giant Apollo s...,2025-07-17T13:00:10Z,"{'negative': 0.025278883, 'neutral': 0.7703285...",neutral,"[0.0006281165988184512, -0.07895796000957489, ..."
2,Gizmodo.com,Truth Social Needs Its Own Grok. Trump Has the...,A MAGA-bot could be on the way.,2025-07-17T13:15:25Z,"{'negative': 0.06825258, 'neutral': 0.04631966...",positive,"[-0.0829867348074913, -0.03853766992688179, -0..."
3,Gizmodo.com,"As Tesla’s Sales Dwindle, a Critical Revenue S...",Tesla needs all the help it can get.,2025-07-23T13:10:27Z,"{'negative': 0.23368655, 'neutral': 0.01441254...",positive,"[-0.04854635149240494, 0.03438419848680496, 0...."
4,Gizmodo.com,Studies Show AI Models Love to Share With One ...,Sharing is caring.,2025-07-30T18:10:01Z,"{'negative': 0.08102807, 'neutral': 0.02253392...",positive,"[-0.05139394477009773, -0.043352656066417694, ..."


In [ ]:
# Ignore __pycache__ and .pyc files in git and remove them from tracking
with open('.gitignore', 'a') as f:
    f.write('\n__pycache__/\n*.pyc\n')

# Remove already tracked __pycache__ and .pyc files from git
!git rm --cached -r **/__pycache__/
!git rm --cached '*.pyc'
!git commit -m 'Remove __pycache__ and .pyc files from tracking and update .gitignore'

rm 'ingest/__pycache__/news_api.cpython-311.pyc'
rm 'models/__pycache__/finbert_model.cpython-311.pyc'
fatal: pathspec '*.pyc' did not match any files
fatal: pathspec '*.pyc' did not match any files
[main 71be9ae] Remove __pycache__ and .pyc files from tracking and update .gitignore
 Committer: Sanjan Kondaparthi <skondaparthi@sanjans-air.lan>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 2 files changed, 0 insertions(+), 0 deletions(-)
 delete mode 100644 ingest/__pycache__/news_api.cpython-311.pyc
 delete mode 100644 models/__pycache__/finbert_model.cpython-311.pyc
[main 71be9ae] Remove __pycache__ and .pyc files from tracking and updat

rm 'ingest/__pycache__/news_api.cpython-311.pyc'
rm 'models/__pycache__/finbert_model.cpython-311.pyc'
fatal: pathspec '*.pyc' did not match any files
fatal: pathspec '*.pyc' did not match any files
[main 71be9ae] Remove __pycache__ and .pyc files from tracking and update .gitignore
 Committer: Sanjan Kondaparthi <skondaparthi@sanjans-air.lan>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 2 files changed, 0 insertions(+), 0 deletions(-)
 delete mode 100644 ingest/__pycache__/news_api.cpython-311.pyc
 delete mode 100644 models/__pycache__/finbert_model.cpython-311.pyc
[main 71be9ae] Remove __pycache__ and .pyc files from tracking and updat

OSError: [Errno 5] Input/output error

In [ ]:
# Virtual environment is now properly ignored via .gitignore
# All important files have been committed successfully!
print("✅ Project setup complete!")
print("✅ Virtual environment isolated and ignored by git")
print("✅ Dependencies managed via requirements.txt")
print("✅ Ready for development and sharing!")

fatal: pathspec 'finsentenv/' did not match any files
On branch vslocal
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   FinSent.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.gitignore
	finsentenv/
	models/faiss_model.py
	requirements.txt

no changes added to commit (use "git add" and/or "git commit -a")
On branch vslocal
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   FinSent.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.gitignore
	finsentenv/
	models/faiss_model.py
	requirements.txt

no changes added to commit (use "git add" and/or "git commit -a")
